In [77]:
from getFEC import *
import warnings

In [16]:
def req_start_url_schedule_a(employer, api_key, year=2016, page=1):
    firm = employer.replace(" ", "%20")
    stem = "https://api.open.fec.gov/v1/schedules/schedule_a/?per_page=100&sort=contribution_receipt_date&"
    end = "contributor_employer={}&api_key={}&two_year_transaction_period={}&page={}".format(firm, api_key, year, page)
    url = stem+end
    return url

req_start_url_schedule_a("Goldman Sachs", api_key, "2008")

'https://api.open.fec.gov/v1/schedules/schedule_a/?per_page=100&sort=contribution_receipt_date&contributor_employer=Goldman%20Sachs&api_key=AcVZVok6ajttkDVo4aQBvIyfgMLC2dEPjvtDrAUB&two_year_transaction_period=2008&page=1'

In [23]:
def req_start_url_schedule_a(employer, year, api_key=api_key):
    firm = employer.replace(" ", "%20")
    url = "https://api.open.fec.gov/v1/schedules/schedule_a/?sort=contribution_receipt_date&per_page=100&"+\
    "contributor_employer={}&two_year_transaction_period={}&api_key={}".format(firm, year, api_key)
    return url


In [22]:
def req_loop_url_schedule_a(start_url, last_indexes):
    """
    ::start_url = schedule_a url created by req_start_url_schedule_a
    ::last_indexes: a list including the following:
            'last_index' returned from the prior url
            'last_contribution_receipt_date' from prior url
    """
    
    end = "&last_index={}&last_contribution_receipt_date={}".format(last_indexes[0], last_indexes[1])
    url = start_url+end
    return url
    

In [24]:
def get_last_index_contrib(data):
    last_index = data['pagination']['last_indexes']['last_index']
    last_contrib = data['pagination']['last_indexes']['last_contribution_receipt_date']
    return [last_index, last_contrib]

In [27]:
def still_results(data):
    results = data['results']
    return len(results)


In [49]:
start_url = req_start_url_schedule_a("Goldman Sachs", "2008", api_key)
data = get_url(start_url)
print(data['pagination'])
last_indexes = get_last_index_contrib(data)
next_url = req_loop_url_schedule_a(start_url, last_indexes)
print(next_url)
still_results(data)

{'count': 4261, 'pages': 43, 'last_indexes': {'last_index': '4032620071076122159', 'last_contribution_receipt_date': '2007-02-15T00:00:00'}, 'per_page': 100}
https://api.open.fec.gov/v1/schedules/schedule_a/?sort=contribution_receipt_date&per_page=100&contributor_employer=Goldman%20Sachs&two_year_transaction_period=2008&api_key=AcVZVok6ajttkDVo4aQBvIyfgMLC2dEPjvtDrAUB&last_index=4032620071076122159&last_contribution_receipt_date=2007-02-15T00:00:00


100

In [111]:
def get_schedule_a_employer_year(employer, year, api_key=api_key):
    start_url = req_start_url_schedule_a(employer, year, api_key)
    data = get_url(start_url)
    if still_results(data) <= 0:
        warnings.warn('WARNING: no data found for requested year')
        return
    
    

    #Repeated Internal Function
    def write_json_to_csv():
        print("[*] getting FEC SCHEDULE A results for {} in {}...page {} of {}".format(employer, year, page, (pages)))
        filename = "{}_{}_schedule_a_{}.csv".format(year, employer.replace(" ", "_"), page)
        write_csv_json_dict(data['results'], filename)
    
    #Looping
    page = 1
    pages = get_pages(data)
    results_count = still_results(data)
    last_indexes = get_last_index_contrib(data)
    
    #First Page
    write_json_to_csv()
    
    while results_count > 0:
        page +=1
        #Last Indexes From the Most Recent Data
        last_indexes = get_last_index_contrib(data)
        next_url = req_loop_url_schedule_a(start_url, last_indexes)
        data = get_url(next_url)
        results_count = still_results(data)
        try:
            if results_count > 0:
                write_json_to_csv()
            else:
                pass
        except:
            if results_count <= 0:
                pass
            else:
                print("[*] ERROR: there are still api results, unknown error in writing json...")
                break
        
    else:
        print("[*] SUCCESS: collected all api results requested.")
        pass

In [112]:
get_schedule_a_employer_year("Goldman Sachs", "2008")

[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 1 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 2 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 3 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 4 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 5 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 6 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 7 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 8 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 9 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 10 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 11 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 12 of 43
[*] getting FEC SCHEDULE A results for Goldman Sachs in 2008...page 13 of 43
[*] gett

In [113]:
def collapse_csvs(company, schedule_type, year=None):
    company = str(company).replace(" ", "_")
    schedule_type = str(schedule_type).replace(" ", "_")
    
    if year is None:
        file_type = "{}_{}".format(company, schedule_type)
        print(file_type)
        filenames = glob('*{}*'.format(file_type))
    else:
        file_type = "{}_{}_{}".format(year, company, schedule_type)
        print(file_type)
        filenames = glob('*{}*'.format(file_type))
    
    assert len(filenames) > 0, "No matching file types, check filename input"
    print("[*] collapsing {} csv files...".format(len(filenames)))
    combined_csv = pd.concat( [ pd.read_csv(f) for f in filenames ] )
    dedupe_csv = combined_csv.drop_duplicates()
    dedupe_csv.to_csv("pandas_test_GS2008_dedupe.csv", index=False)
    print("[*] done")

collapse_csvs("Goldman Sachs", "schedule a")
#collapse_csvs("Goldman Sachs", "schedule a", "2010")

Goldman_Sachs_schedule_a
[*] collapsing 43 csv files...
[*] done


In [108]:
get_schedule_a_employer_year("Goldman Sachs", "2015")

https://api.open.fec.gov/v1/schedules/schedule_a/?sort=contribution_receipt_date&per_page=100&contributor_employer=Goldman%20Sachs&two_year_transaction_period=2015&api_key=AcVZVok6ajttkDVo4aQBvIyfgMLC2dEPjvtDrAUB
{'results': [], 'pagination': {'count': 0, 'pages': 0, 'last_indexes': None, 'per_page': 100}, 'api_version': '1.0'}
pass


/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: WARNING: no data found for requested year
